#### Convolutional Neural Network (CNN) for classifying the CIFAR-10 Dataset

In [8]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [92]:
num_epochs = 1
batch_size = 10
learning_rate = 0.01

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [1.0, 1.0, 1.0])]
)

In [12]:
train_dataset = torchvision.datasets.CIFAR10(root='./datasets', download=True, transform=transform, train=True)
test_dataset = torchvision.datasets.CIFAR10(root='./datasets', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size)

100%|██████████| 170M/170M [01:56<00:00, 1.46MB/s] 


Extracting ./datasets\cifar-10-python.tar.gz to ./datasets


In [50]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truch')

In [51]:
class ConNeuNet(nn.Module):
    def __init__(self):
        super(ConNeuNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.dense1 = nn.Linear(16*5*5, 150)
        self.dense2 = nn.Linear(150, 100)
        self.dense3 = nn.Linear(100, 50)
        self.dense4 = nn.Linear(50, 10)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*5*5)
        out = F.relu(self.dense1(out))
        out = F.relu(self.dense2(out))
        out = F.relu(self.dense3(out))
        out = self.dense4(out)
        return out
        


In [91]:
model = ConNeuNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for index, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (index + 1) % 1000 == 0:
            print(f'epoch: {epoch + 1}, step = {index+1}, loss = {loss}')
            

epoch: 1, step = 10, loss = 1.1841310262680054
epoch: 1, step = 20, loss = 2.0186545848846436
epoch: 1, step = 30, loss = 2.1949727535247803
epoch: 1, step = 40, loss = 1.6928982734680176
epoch: 1, step = 50, loss = 2.9375038146972656
epoch: 1, step = 60, loss = 1.760434865951538
epoch: 1, step = 70, loss = 2.7317724227905273
epoch: 1, step = 80, loss = 1.8725275993347168
epoch: 1, step = 90, loss = 1.79835045337677
epoch: 1, step = 100, loss = 1.9946645498275757
epoch: 1, step = 110, loss = 1.7887320518493652
epoch: 1, step = 120, loss = 1.0279408693313599
epoch: 1, step = 130, loss = 2.2855825424194336
epoch: 1, step = 140, loss = 2.2730889320373535
epoch: 1, step = 150, loss = 1.790522575378418
epoch: 1, step = 160, loss = 1.550828218460083
epoch: 1, step = 170, loss = 1.328562617301941
epoch: 1, step = 180, loss = 2.9793472290039062
epoch: 1, step = 190, loss = 1.9463212490081787
epoch: 1, step = 200, loss = 1.840070366859436
epoch: 1, step = 210, loss = 1.7370672225952148
epoch: 1

In [98]:
correct = 0
total = 0
for index, (images, labels) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    _, predictions = torch.max(predictions, dim=1)
    total += len(predictions)
    correct += (predictions == labels).sum().item()
print(f'accuracy : {correct / total * 100}')

accuracy : 28.27
